#  IBM Data Science Capstone Project 
## Planning Travel and Tours in New York City's Broadway Theatre District
### Comparison of Hotels, Restaurants, and Theaters in Manhattan, NY for a Broadway Theater Trip
### Brian Vineyard 

### Business Problem


**Business Problem**: A travel service is planning to offer Broadway trips with the best options in New York City, NY to include on their package. They want to build a package with varying hotels, and a list of recommended restaurants. 

Their clients are mostly interested in Broadway musicals and plays, so this project will focus on finding the best hotels and restaurants near the Broadway theater district.

![Broadway](77.jpg)

**Key Questions to Answer**

- What are the best theaters to catch the top Broadway shows?
- What hotels close to the Theater District have the best ratings and room rates?
- What are the best nearby restaurants?


**Foursquare API study of following city**:

- Broadway Theatre District, New York City, NY

Example search criteria: Hotels, Restaurants, Museums, Live Shows

**Project Details**: 

This project uses the following tools and technology:

- [Foursquare Places API](https://https://enterprise.foursquare.com/products/places)
- [2014 New York City Neighborhood Names](https://geo.nyu.edu/catalog/nyu_2451_34572)
- [BeautifulSoup](https://beautiful-soup-4.readthedocs.io/en/latest/)


The goal is to find the best area of the city to hold tours, based on our findings.

**Data Sources and Description**:

This project will use data for the latitude and longitude of zipcodes located in the following city:

- New York, NY


The coordinates will be passed to the Foursquare Places API, for querying New York City neighborhoods for venues such as:

- **Restaurants**
    - Liebman's Deli, La Morada, Royal 35 Steakhouse, Le Benardin
- **Hotels**
    - Gramercy Park, Life Hotel Nomad, Mansfield Hotel
- **Fun**
    - Museums: Metropoliton Museum of Art, 9/11 Memorial, American Museum of Natural History
    - Top Tourist Destinations: Empire State Building, Madison Square Garden, Statue of Liberty, Times Square
    - Broadway Productions: Hamilton, The Lion King, Wicked, Moulin Rouge, Phantom of the Opera
    - Sightseeing Cruises: Circle Line Full Island, Classic Harbor Lines, Zephyr Yacht, Liberty Cruises
    - Parks: Central Park, Washington Square Park, Fort Tryon Park, Bronx Zoo, 
    
 ### Table of Contents
 1. [Import Libraries, Foursquare Credentials](#import-1)
 2. [New York's Manhattan Borough Neighborhood Data](#borough-2)
   - [Map of Neighborhoods in New York City's Five Boroughs](#borough-map)
   - [Map of Neighborhoods in NYC's Manhattan Borough](#neighor-map)
 3. [Broadway Theatres and Current Productions](#broadway-3)
   - [Map of Broadway Theatre Locations](#broadway-map)
 4. [Hotels Near the Broadway Theatre District](#hotels-4)
   - [Map of Hotels Near Broadway Theatres](#hotels-map)
 5. [Restaurants Near the Broadway Theatre District](#restaurants-5)
   - [Map of Restaurants Near Broadway Theatres](#rest_map)
 6. [Conclusions and Findings](#conclusions)

## 1. Import Libraries, Foursquare Credentials, and Neighborhood Data <a name="import-1"></a>

### Import Libraries ###

In [284]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files
import pickle

from geopy.geocoders import Nominatim # Convert addresses into GPS coordinates
from bs4 import BeautifulSoup

import requests # library to handle requests

from pandas.io.json import json_normalize # transform JSON file into a Pandas dataframe
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library
print('Libraries imported.')

Libraries imported.


### Load Foursquare Parameters ###

In [285]:
# Loads the Foursquare credentials for logging in to the API

file = open('credentials2.p', 'rb')
credentials = pickle.load(file)
file.close()
CLIENT_ID = credentials['CLIENT_ID']
CLIENT_SECRET = credentials['CLIENT_SECRET']
VERSION = '20180604'
LIMIT = 30

## 2. New York's Manhattan Borough Neighborhood Data <a name="borough-2"></a>

### Import Neighborhood Data ###

New York City has a total of 5 boroughs and 306 neighborhoods. We will be concentrating on the Manhattan borough, where Broadway and the Theatre District are located. We will start with a larger dataset, however, with the latitude and longitude of each of NYC's neighborhoods.

We will pull in the data as a JSON file, then load that into a pandas dataframe for analysis.

#### Open  and explore the JSON file for New York City Neighborhood Names ####

In [286]:
with open('nyu-2451-34572-geojson.json') as json_data:
    newyork_data = json.load(json_data)

In [287]:
# View the data from the JSON file

newyork_data

{'type': 'FeatureCollection',
 'totalFeatures': 306,
 'features': [{'type': 'Feature',
   'id': 'nyu_2451_34572.1',
   'geometry': {'type': 'Point',
    'coordinates': [-73.84720052054902, 40.89470517661]},
   'geometry_name': 'geom',
   'properties': {'name': 'Wakefield',
    'stacked': 1,
    'annoline1': 'Wakefield',
    'annoline2': None,
    'annoline3': None,
    'annoangle': 0.0,
    'borough': 'Bronx',
    'bbox': [-73.84720052054902,
     40.89470517661,
     -73.84720052054902,
     40.89470517661]}},
  {'type': 'Feature',
   'id': 'nyu_2451_34572.2',
   'geometry': {'type': 'Point',
    'coordinates': [-73.82993910812398, 40.87429419303012]},
   'geometry_name': 'geom',
   'properties': {'name': 'Co-op City',
    'stacked': 2,
    'annoline1': 'Co-op',
    'annoline2': 'City',
    'annoline3': None,
    'annoangle': 0.0,
    'borough': 'Bronx',
    'bbox': [-73.82993910812398,
     40.87429419303012,
     -73.82993910812398,
     40.87429419303012]}},
  {'type': 'Feature',
 

Notice how all the relevant data is in the features key, which is basically a list of the neighborhoods. So, let's define a new variable that includes this data.

In [288]:
neighborhoods_data = newyork_data['features']

Let's take a look at the first item in this list.

In [289]:
neighborhoods_data[0]

{'type': 'Feature',
 'id': 'nyu_2451_34572.1',
 'geometry': {'type': 'Point',
  'coordinates': [-73.84720052054902, 40.89470517661]},
 'geometry_name': 'geom',
 'properties': {'name': 'Wakefield',
  'stacked': 1,
  'annoline1': 'Wakefield',
  'annoline2': None,
  'annoline3': None,
  'annoangle': 0.0,
  'borough': 'Bronx',
  'bbox': [-73.84720052054902,
   40.89470517661,
   -73.84720052054902,
   40.89470517661]}}

#### Tranform the data into a pandas dataframe ####

In [290]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

Loop through the data and load the dataframe one row at a time

In [291]:
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

In [292]:
# Look at the first five rows of data
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [293]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 5 boroughs and 306 neighborhoods.


###  Map of Neighborhoods in the New York City' Five Boroughs <a name="borough-map"></a>

#### Get the New York City latitude and longitude coordinates from geopy ####

In [294]:
address = 'New York City, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of New York City are 40.7127281, -74.0060152.


**Build map of New York City in Folium, with neighborhoods in the five boroughs indicated with blue markers**

In [295]:
# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

**Map of New York City Neighborhoods in Five Boroughs: Manhattan, The Bronx, Brooklyn, Queens, and Staten Island**

### Map of Neighborhoods in NYC's Manhattan Borough <a name="neighbor-map"></a>

New York City has five boroughs:
    
    - Manhattan
    - The Bronx
    - Brooklyn
    - Queens
    - Staten Island
    
We will be looking more closely at neighborhoods in Manhattan, as this is where Broadway's Theatre District is located.

Let's take the Manhattan data and build that into a new dataframe for that borough.

In [296]:
manhattan_data_df = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
manhattan_data_df.head()

,Borough,Neighborhood,Latitude,Longitude
0,Manhattan,Marble Hill,40.876551,-73.910660
1,Manhattan,Chinatown,40.715618,-73.994279
2,Manhattan,Washington Heights,40.851903,-73.936900
3,Manhattan,Inwood,40.867684,-73.921210
4,Manhattan,Hamilton Heights,40.823604,-73.949688


Let get the size of the Manhattan data frame:

In [297]:
manhattan_data_df.shape

(40, 4)

From our results, we can see there are 40 neighborhoods within Manhattan.

Let's take a look at all the values in the dataframe:

In [298]:
manhattan_data_df

,Borough,Neighborhood,Latitude,Longitude
0,Manhattan,Marble Hill,40.876551,-73.910660
1,Manhattan,Chinatown,40.715618,-73.994279
2,Manhattan,Washington Heights,40.851903,-73.936900
3,Manhattan,Inwood,40.867684,-73.921210
4,Manhattan,Hamilton Heights,40.823604,-73.949688
5,Manhattan,Manhattanville,40.816934,-73.957385
6,Manhattan,Central Harlem,40.815976,-73.943211
7,Manhattan,East Harlem,40.792249,-73.944182
8,Manhattan,Upper East Side,40.775639,-73.960508
9,Manhattan,Yorkville,40.775930,-73.947118


### Build map of Manhattan neighborhoods from latitude and longitude coordinates ###

Close to Times Square, there is a TKTS ticket booth that sells ticket to all the Broadway shows.

Let's set our address in Nominatim for the TKTS ticket booth location get the coordinates.

Let's draw a map using Folium, marking the TKTS booth near Times Square as a central location in the Theatre District, with green markers for the neighborhoods in Manhattan.

In [299]:
address = 'Broadway at, W 47th St, New York, NY 10036'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = 40.7591855
longitude = -73.9848361
print('The geograpical coordinates of the TKTS booth near Times Square are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of the TKTS booth near Times Square are 40.7591855, -73.9848361.


In [300]:
# create map of the Theatre District in Manhattan using latitude and longitude values
map_manhattan = folium.Map(location=[latitude, longitude], zoom_start=11)

# add a red circle marker to represent the TKTS ticket booth
folium.features.CircleMarker(
    [latitude, longitude],
    radius=5,
    color='red',
    popup='TKTS ticket booth in Times Square',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(map_manhattan)

# add markers to map
for lat, lng, label in zip(manhattan_data['Latitude'], manhattan_data['Longitude'], manhattan_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='green',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_manhattan)  
    
map_manhattan

**Manhattan, New York with markers for the 40 neighborhoods, and the TKTS booth near Times Square marked in red.**

## 3. Broadway Theatres and Current Productions <a name="broadway-3"></a>

## Theatres on Broadway - Maps and Analysis ##

Now we have some hotels in our dataset, let's have a look at the 40 theatres currently showing Broadway productions.

**Some of the most popular shows at the time of this project are**:
![Shows](shows.jpg)

Let's start by using BeautifulSoup to parse a list of the theatres and currently running shows from the following Wikipedia page:
https://en.wikipedia.org/wiki/Broadway_theatre

### Build pandas dataframe with Theatre Data scraped from a Broadway Wiki Page using BeautifulSoup

In [301]:
# Import Pandas and Beautiful Soup libraries
import pandas as pd
import urllib.request
from bs4 import BeautifulSoup
import requests

In [302]:
# Use Beautiful Soup to get the theatre data from the wiki page
sourcelink = 'https://en.wikipedia.org/wiki/Broadway_theatre'
source = requests.get(sourcelink).text
soup = BeautifulSoup(source, 'html.parser')
print(soup.prettify())

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   Broadway theatre - Wikipedia
  </title>
  <script>
   document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgMonthNamesShort":["","Jan","Feb","Mar","Apr","May","Jun","Jul","Aug","Sep","Oct","Nov","Dec"],"wgRequestId":"XjrEPgpAADoAAF-vfJ0AAAEG","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"Broadway_theatre","wgTitle":"Broadway theatre","wgCurRevisionId":938745947,"wgRevisionId":938745947,"wgArticleId":725252,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Webarchive template wayback links","Articles with short de

In [303]:
# Build dataframe with theatre data

Theatre_Data_df = pd.DataFrame({'Theatre':'',
                          'Address':'',
                          'Capacity':int(),
                          'OwnerOperator':'',
                          'CurrentProduction':'',
                          'Type':'',
                          'Opening':'',
                          'Closing':'',
}, 
             index=[1])

In [304]:
# Build theatres table from matching table on the theatres wiki page and view the HTML code
theatres_table = soup.find('table', {'class':'wikitable sortable'})
theatres_table

<table class="wikitable sortable" style="font-size:96%;">
<tbody><tr>
<th>Theatre
</th>
<th>Address
</th>
<th>Capacity
</th>
<th>Owner/Operator
</th>
<th>Current production
</th>
<th>Type
</th>
<th>Opening
</th>
<th>Closing
</th></tr>
<tr>
<td><a href="/wiki/Al_Hirschfeld_Theatre" title="Al Hirschfeld Theatre">Al Hirschfeld Theatre</a></td>
<td>W. 45th St. <small>(No. 302)</small></td>
<td>1424</td>
<td><a href="/wiki/Jujamcyn_Theaters" title="Jujamcyn Theaters">Jujamcyn Theaters</a></td>
<td><i><a href="/wiki/Moulin_Rouge!_(musical)" title="Moulin Rouge! (musical)">Moulin Rouge!</a></i><sup class="reference" id="cite_ref-47"><a href="#cite_note-47">[46]</a></sup></td>
<td>Musical</td>
<td><span style="display:none">2019-07-25</span>July 25, 2019</td>
<td>Open-ended
</td></tr>
<tr>
<td><a href="/wiki/Ambassador_Theatre_(New_York_City)" title="Ambassador Theatre (New York City)">Ambassador Theatre</a></td>
<td>W. 49th St. <small>(No. 219)</small></td>
<td>1125</td>
<td><a href="/wiki/Th

### Loop through the theatres table and load our dataframe row by row.

In [305]:
# Initialize values for dataframe fields
Theatre = 0
Address = 0
Capacity = 0
OwnerOperator = 0
CurrentProduction = 0
Type = 0
Opening = 0
Closing = 0

for tr in theatres_table.find_all('tr'):
    i = 0
    for td in tr.find_all('td'):
        if i == 0:
            Theatre = td.text
            i = i + 1
        elif i == 1:
            Address = td.text
            i = i + 1
        elif i == 2:
            Capacity = td.text
            i = i + 1
        elif i == 3:
            Owner_Operator = td.text
            i = i + 1
        elif i == 4:
            CurrentProduction = td.text
            i = i + 1
        elif i == 5:
            Type = td.text
            i = i + 1
        elif i == 6:
            Opening = td.text
            i = i + 1
        elif i == 7:
            Closing = td.text.strip('\n').replace(']','')
            i = i + 1        
            Theatre_Data_df = Theatre_Data_df.append({'Theatre': Theatre,'Address': Address,'Capacity': Capacity, 'OwnerOperator': Owner_Operator, 'CurrentProduction': CurrentProduction, 'Type': Type, 'Opening': Opening, 'Closing': Closing},ignore_index=True)

In [306]:
# Let's look at the data in our theatres dataframe
Theatre_Data_df

,Theatre,Address,Capacity,OwnerOperator,CurrentProduction,Type,Opening,Closing
0,,,0,,,,,
1,Al Hirschfeld Theatre,W. 45th St. (No. 302),1424,Jujamcyn Theaters,Moulin Rouge![46],Musical,"2019-07-25July 25, 2019",Open-ended
2,Ambassador Theatre,W. 49th St. (No. 219),1125,Shubert Organization,Chicago,Musical,"1996-11-14November 14, 1996",Open-ended
3,American Airlines Theatre,W. 42nd St. (No. 227),740,Roundabout Theatre Company,A Soldier's Play[47],Play,"2020-01-21January 21, 2020","2020-03-15March 15, 2020*"
4,August Wilson Theatre,W. 52nd St. (No. 245),1228,Jujamcyn Theaters,Mean Girls,Musical,"2018-04-08April 8, 2018",Open-ended
5,Belasco Theatre,W. 44th St. (No. 111),1018,Shubert Organization,Girl from the North Country[48],Musical,"2020-03-05March 5, 2020*",Open-ended
6,Bernard B. Jacobs Theatre,W. 45th St. (No. 242),1078,Shubert Organization,Company[49],Musical,"2020-03-22March 22, 2020*",Open-ended
7,Booth Theatre,W. 45th St. (No. 222),766,Shubert Organization,Who's Afraid of Virginia Woolf?[50],Play,"2020-04-09April 9, 2020*","2020-08-02August 2, 2020"
8,Broadhurst Theatre,W. 44th St. (No. 235),1186,Shubert Organization,Jagged Little Pill[51],Musical,"2019-12-05December 5, 2019",Open-ended
9,Broadway Theatre,W. 53rd St & Broadway (No. 1681),1761,Shubert Organization,West Side Story[52],Musical,"2020-02-20February 20, 2020*",Open-ended


In [307]:
# Let's drop the empty first row
Theatre_Data_df.drop([0], inplace=True)

In [308]:
# Let's drop some of the columns that aren't needed
Theatre_Data_df.drop(['OwnerOperator','Type','Opening','Closing'],axis=1)
Theatre_Data_df

,Theatre,Address,Capacity,OwnerOperator,CurrentProduction,Type,Opening,Closing
1,Al Hirschfeld Theatre,W. 45th St. (No. 302),1424,Jujamcyn Theaters,Moulin Rouge![46],Musical,"2019-07-25July 25, 2019",Open-ended
2,Ambassador Theatre,W. 49th St. (No. 219),1125,Shubert Organization,Chicago,Musical,"1996-11-14November 14, 1996",Open-ended
3,American Airlines Theatre,W. 42nd St. (No. 227),740,Roundabout Theatre Company,A Soldier's Play[47],Play,"2020-01-21January 21, 2020","2020-03-15March 15, 2020*"
4,August Wilson Theatre,W. 52nd St. (No. 245),1228,Jujamcyn Theaters,Mean Girls,Musical,"2018-04-08April 8, 2018",Open-ended
5,Belasco Theatre,W. 44th St. (No. 111),1018,Shubert Organization,Girl from the North Country[48],Musical,"2020-03-05March 5, 2020*",Open-ended
6,Bernard B. Jacobs Theatre,W. 45th St. (No. 242),1078,Shubert Organization,Company[49],Musical,"2020-03-22March 22, 2020*",Open-ended
7,Booth Theatre,W. 45th St. (No. 222),766,Shubert Organization,Who's Afraid of Virginia Woolf?[50],Play,"2020-04-09April 9, 2020*","2020-08-02August 2, 2020"
8,Broadhurst Theatre,W. 44th St. (No. 235),1186,Shubert Organization,Jagged Little Pill[51],Musical,"2019-12-05December 5, 2019",Open-ended
9,Broadway Theatre,W. 53rd St & Broadway (No. 1681),1761,Shubert Organization,West Side Story[52],Musical,"2020-02-20February 20, 2020*",Open-ended
10,Brooks Atkinson Theatre,W. 47th St. (No. 256),1094,Nederlander Organization,Six[53],Musical,"2020-03-12March 12, 2020*",Open-ended


Let's download the data into a .csv file for manual processing in Excel. We will clean up the format of the address data.

In [309]:
Theatre_Data_df.to_csv(r'C:\Users\brian\BroadwayTheatres.csv')

In [310]:
# Build a new dataframe with cleaner theatre data
Column_Names = ['Theatre','Address','City','State','ZipCode','Latitude','Longitude','CurrentProduction','Type']
Theatre_Data_df4 = pd.DataFrame(columns=Column_Names)

In [311]:
Theatre_Data_df4 = pd.read_csv('BroadwayTheatres2.csv',dtype={'Latitude': float,'Longitude': float})
Theatre_Data_df4

,Theatre,Address,City,State,Capacity,CurrentProduction,Type,Latitude,Longitude
0,Al Hirschfeld Theatre,302 West 45th St.,New York,NY,1424,Moulin Rouge![46],Musical,40.759253,-73.989211
1,Ambassador Theatre,219 West 49th St.,New York,NY,1125,Chicago,Musical,40.761236,-73.984990
2,American Airlines Theatre,227 West 42nd St.,New York,NY,740,A Soldier's Play[47],Play,40.757156,-73.988119
3,August Wilson Theatre,245 West 52nd St.,New York,NY,1228,Mean Girls,Musical,40.763373,-73.984193
4,Belasco Theatre,111 West 44th St.,New York,NY,1018,Girl from the North Country[48],Musical,40.756644,-73.983801
5,Bernard B. Jacobs Theatre,242 West 45th St.,New York,NY,1078,Company[49],Musical,40.758608,-73.987741
6,Booth Theatre,222 West 45th St.,New York,NY,766,Who's Afraid of Virginia Woolf?[50],Play,40.758373,-73.987090
7,Broadhurst Theatre,235 West 44th St.,New York,NY,1186,Jagged Little Pill[51],Musical,40.758269,-73.987617
8,Broadway Theatre,1681 West 53rd St & Broadway,New York,NY,1761,West Side Story[52],Musical,40.764457,-73.985746
9,Brooks Atkinson Theatre,256 West 47th St.,New York,NY,1094,Six[53],Musical,40.759975,-73.986966


### Map of Broadway Theatre Locations <a name="broadway-map"></a>

In [312]:
# Setting the latitude and longitude of Broadway

broadway_latitude = 40.75659 # neighborhood latitude value
broadway_longitude = -73.98626, # neighborhood longitude value

neighborhood_name = 'Broadway'

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               broadway_latitude, 
                                                               broadway_longitude))

Latitude and longitude values of Broadway are 40.75659, (-73.98626,).


In [313]:
# create map of the Broadway theatre district using latitude and longitude values
map_broadway = folium.Map(location=[broadway_latitude, broadway_longitude], zoom_start=14)

# add markers to map
for lat, lng, label in zip(Theatre_Data_df4['Latitude'], Theatre_Data_df4['Longitude'], Theatre_Data_df4['Theatre']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='black',
        fill=True,
        fill_color='purple',
        fill_opacity=0.7,
        parse_html=False).add_to(map_broadway) 

map_broadway

**Map Showing Locations of 40 Broadway Theatres in Manhattan, New York City**

#### Theatre Location Analysis ####

Looking at the mapped results, we see that the greatest concentration of theatres appears clustered near 44th and 45th streets between 7th Avenue and 8th Avenue.

This area is looking like a great spot to find local restaurants and hotels.

Let's set our starting point at the Richard Rodgers Theatre where *Hamilton* is currently playing.

This theatre is also located near the center of our cluster:

![Richard Rodgers Theatre](RichardRodgersMap.jpg)


**Google Maps street level view of the intersection of West 47th Street and 7th Avenue from the TKTS booth location**:
![TKTS Booth View](TKTSView.jpg)

## 4. Hotels Near the Broadway Theatre District <a name="hotels-4"></a>

### Search with the Foursquare API for Hotels near the Richard Rodgers Theatre

We will use the address of Richard Rodgers Theatre as our starting point as we look for nearby hotels using the Foursquare API.

Let's get the hotel's latitude and longitude values.

In [314]:
# Set address of Richard Rodgers Theatre as our starting point
address = '226 West 46th Street, New York, NY'
geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)
    

40.7590309 -73.98674788284762


We can use Foursquare to view hotels within a given radius of the Richard Rodgers Theatre.

In [315]:
search_query = 'Hotel'
radius = 500
print(search_query + ' .... OK!')

Hotel .... OK!


In [316]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=QQ5ZYS0QBI2WYIMIAZZUFEZJEV10V25IVVCFTW3EB550JISX&client_secret=3CJR0S30CC0RXHCOYTX5XUG1RCMVGA5MKV00WCOEQD41ESE1&ll=40.7590309,-73.98674788284762&v=20180604&query=Hotel&radius=500&limit=30'

Let's get the JSON data and set that into a variable called 'results'

In [317]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e3c5d591a4b0a001b6397da'},
 'response': {'venues': [{'id': '4b4bbe3ff964a52016a626e3',
    'name': 'Hotel Edison',
    'location': {'address': '228 W 47th St',
     'crossStreet': 'at 8th Ave',
     'lat': 40.75966639038906,
     'lng': -73.98608770066558,
     'labeledLatLngs': [{'label': 'display',
       'lat': 40.75966639038906,
       'lng': -73.98608770066558}],
     'distance': 90,
     'postalCode': '10036',
     'cc': 'US',
     'city': 'New York',
     'state': 'NY',
     'country': 'United States',
     'formattedAddress': ['228 W 47th St (at 8th Ave)',
      'New York, NY 10036',
      'United States']},
    'categories': [{'id': '4bf58dd8d48988d1fa931735',
      'name': 'Hotel',
      'pluralName': 'Hotels',
      'shortName': 'Hotel',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/travel/hotel_',
       'suffix': '.png'},
      'primary': True}],
    'venuePage': {'id': '501560524'},
    'referralId': 'v-1581014290',

In [318]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

,categories,delivery.id,delivery.provider.icon.name,delivery.provider.icon.prefix,delivery.provider.icon.sizes,delivery.provider.name,delivery.url,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.neighborhood,location.postalCode,location.state,name,referralId,venuePage.id
0,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'Hotel', 'pluralName': 'Hotels', 'shortName': 'Hotel', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/travel/hotel_', 'suffix': '.png'}, 'primary': True}]",NaN,NaN,NaN,NaN,NaN,NaN,False,4b4bbe3ff964a52016a626e3,228 W 47th St,US,New York,United States,at 8th Ave,90,"[228 W 47th St (at 8th Ave), New York, NY 10036, United States]","[{'label': 'display', 'lat': 40.75966639038906, 'lng': -73.98608770066558}]",40.759666,-73.986088,NaN,10036,NY,Hotel Edison,v-1581014290,501560524
1,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'Hotel', 'pluralName': 'Hotels', 'shortName': 'Hotel', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/travel/hotel_', 'suffix': '.png'}, 'primary': True}]",NaN,NaN,NaN,NaN,NaN,NaN,False,4d7ad1263fbf6dcbb60b6423,132 W 47th St,US,New York,United States,btwn 6th & 7th Ave.,303,"[132 W 47th St (btwn 6th & 7th Ave.), New York, NY 10036, United States]","[{'label': 'display', 'lat': 40.7583581, 'lng': -73.9832623}]",40.758358,-73.983262,NaN,10036,NY,Sanctuary Hotel New York,v-1581014290,NaN
2,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'Hotel', 'pluralName': 'Hotels', 'shortName': 'Hotel', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/travel/hotel_', 'suffix': '.png'}, 'primary': True}]",NaN,NaN,NaN,NaN,NaN,NaN,False,5bc65d8e1822230025fffcb1,310 W 40th St,US,New York,United States,NaN,514,"[310 W 40th St, New York, NY 10018, United States]","[{'label': 'display', 'lat': 40.75620715052, 'lng': -73.99157595205548}]",40.756207,-73.991576,NaN,10018,NY,Aliz Hotel,v-1581014290,NaN
3,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'Hotel', 'pluralName': 'Hotels', 'shortName': 'Hotel', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/travel/hotel_', 'suffix': '.png'}, 'primary': True}]",NaN,NaN,NaN,NaN,NaN,NaN,False,4adbaf34f964a520012a21e3,235 W 46th St,US,New York,United States,NaN,66,"[235 W 46th St, New York, NY 10036, United States]","[{'label': 'display', 'lat': 40.759568887065484, 'lng': -73.98709530548786}]",40.759569,-73.987095,NaN,10036,NY,Paramount Hotel,v-1581014290,33865901
4,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'Hotel', 'pluralName': 'Hotels', 'shortName': 'Hotel', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/travel/hotel_', 'suffix': '.png'}, 'primary': True}]",NaN,NaN,NaN,NaN,NaN,NaN,False,5093c236830214706abb75db,218 W 50th St,US,New York,United States,50th & B'way,332,"[218 W 50th St (50th & B'way), New York, NY 10019, United States]","[{'label': 'display', 'lat': 40.761691026346256, 'lng': -73.98495316332877}]",40.761691,-73.984953,NaN,10019,NY,citizenM Hotel New York Times Square,v-1581014290,NaN


#### Let's look at data for Hotels within 500 meters of the Richard Rodgers Theatre

In [319]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns2 = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered2 = dataframe.loc[:, filtered_columns2]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered2['categories'] = dataframe_filtered2.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered2.columns = [column.split('.')[-1] for column in dataframe_filtered2.columns]

dataframe_filtered2

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,Hotel Edison,Hotel,228 W 47th St,US,New York,United States,at 8th Ave,90,"[228 W 47th St (at 8th Ave), New York, NY 10036, United States]","[{'label': 'display', 'lat': 40.75966639038906, 'lng': -73.98608770066558}]",40.759666,-73.986088,NaN,10036,NY,4b4bbe3ff964a52016a626e3
1,Sanctuary Hotel New York,Hotel,132 W 47th St,US,New York,United States,btwn 6th & 7th Ave.,303,"[132 W 47th St (btwn 6th & 7th Ave.), New York, NY 10036, United States]","[{'label': 'display', 'lat': 40.7583581, 'lng': -73.9832623}]",40.758358,-73.983262,NaN,10036,NY,4d7ad1263fbf6dcbb60b6423
2,Aliz Hotel,Hotel,310 W 40th St,US,New York,United States,NaN,514,"[310 W 40th St, New York, NY 10018, United States]","[{'label': 'display', 'lat': 40.75620715052, 'lng': -73.99157595205548}]",40.756207,-73.991576,NaN,10018,NY,5bc65d8e1822230025fffcb1
3,Paramount Hotel,Hotel,235 W 46th St,US,New York,United States,NaN,66,"[235 W 46th St, New York, NY 10036, United States]","[{'label': 'display', 'lat': 40.759568887065484, 'lng': -73.98709530548786}]",40.759569,-73.987095,NaN,10036,NY,4adbaf34f964a520012a21e3
4,citizenM Hotel New York Times Square,Hotel,218 W 50th St,US,New York,United States,50th & B'way,332,"[218 W 50th St (50th & B'way), New York, NY 10019, United States]","[{'label': 'display', 'lat': 40.761691026346256, 'lng': -73.98495316332877}]",40.761691,-73.984953,NaN,10019,NY,5093c236830214706abb75db
5,"The Algonquin Hotel, Autograph Collection",Hotel,59 W 44th St,US,New York,United States,NaN,503,"[59 W 44th St, New York, NY 10036, United States]","[{'label': 'display', 'lat': 40.7559927, 'lng': -73.9823172}]",40.755993,-73.982317,NaN,10036,NY,45f8e590f964a5203f441fe3
6,The Manhattan at Times Square Hotel,Hotel,790 7th Ave,US,New York,United States,at W 51st St,472,"[790 7th Ave (at W 51st St), New York, NY 10019, United States]","[{'label': 'display', 'lat': 40.76208099999999, 'lng': -73.9828534}]",40.762081,-73.982853,NaN,10019,NY,4bc4f73c0191c9b6d2e8eab1
7,Kimpton Muse Hotel,Hotel,130 W 46th St,US,New York,United States,btwn 6th & 7th Ave,286,"[130 W 46th St (btwn 6th & 7th Ave), New York, NY 10036, United States]","[{'label': 'display', 'lat': 40.757808, 'lng': -73.983764}]",40.757808,-73.983764,NaN,10036,NY,4a9f2f6ff964a520d93c20e3
8,Millennium Broadway Hotel,Hotel,145 W 44th St,US,New York,United States,at Broadway,272,"[145 W 44th St (at Broadway), New York, NY 10036, United States]","[{'label': 'display', 'lat': 40.757073, 'lng': -73.9848}]",40.757073,-73.984800,NaN,10036,NY,4a0215c6f964a5202a711fe3
9,Renaissance New York Times Square Hotel,Hotel,"Two Times Square, 714 Seventh Avenue At W. 48th Street",US,New York,United States,at W 48th St,192,"[Two Times Square, 714 Seventh Avenue At W. 48th Street (at W 48th St), New York, NY 10036, United States]","[{'label': 'display', 'lat': 40.75971, 'lng': -73.984643}]",40.759710,-73.984643,NaN,10036,NY,4a7d0598f964a5205aee1fe3


In [320]:
dataframe_filtered2.name

0     Hotel Edison                                                      
1     Sanctuary Hotel New York                                          
2     Aliz Hotel                                                        
3     Paramount Hotel                                                   
4     citizenM Hotel New York Times Square                              
5     The Algonquin Hotel, Autograph Collection                         
6     The Manhattan at Times Square Hotel                               
7     Kimpton Muse Hotel                                                
8     Millennium Broadway Hotel                                         
9     Renaissance New York Times Square Hotel                           
10    Royalton Hotel                                                    
11    The Belvedere Hotel                                               
12    The Pearl Hotel                                                   
13    Night Hotel Times Square                     

Let's map out the hotels returned in our Foursquare search.

### Map of Hotels Near Broadway Theatres <a name="hotels-map"></a>

First let's show a map of the hotel locations, so we can get a view of where they are in relation to the theatre.

In [321]:
# create map of Manhattan using latitude and longitude values
map_hotels = folium.Map(location=[latitude, longitude], zoom_start=15)

# add a red circle marker to represent the Midtown Manhattan neighborhood center
folium.features.CircleMarker(
    [latitude, longitude],
    radius=5,
    color='red',
    popup='Richard Rodgers Theatre - Home of Hamilton',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(map_hotels)

# add markers to map
for lat, lng, label in zip(dataframe_filtered2['lat'], dataframe_filtered2['lng'], dataframe_filtered2['name']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='purple',
        fill=True,
        fill_color='purple',
        fill_opacity=0.5,
        parse_html=False).add_to(map_hotels)  
    
map_hotels

**Map of Hotels Near the Richard Rodgers Theatre returned by searching for Hotels within a 500 meter radius**

Let's sort the data for the hotels to view the closest ones from the theater, then list the others in ascending order.

In [322]:
dataframe_filtered2.sort_values(by=['distance'])

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
3,Paramount Hotel,Hotel,235 W 46th St,US,New York,United States,NaN,66,"[235 W 46th St, New York, NY 10036, United States]","[{'label': 'display', 'lat': 40.759568887065484, 'lng': -73.98709530548786}]",40.759569,-73.987095,NaN,10036,NY,4adbaf34f964a520012a21e3
0,Hotel Edison,Hotel,228 W 47th St,US,New York,United States,at 8th Ave,90,"[228 W 47th St (at 8th Ave), New York, NY 10036, United States]","[{'label': 'display', 'lat': 40.75966639038906, 'lng': -73.98608770066558}]",40.759666,-73.986088,NaN,10036,NY,4b4bbe3ff964a52016a626e3
29,Riu Plaza Times Square Hotel,Resort,W 46th Street,US,New York,United States,NaN,167,"[W 46th Street, New York, NY, United States]","[{'label': 'display', 'lat': 40.76017910501307, 'lng': -73.98802929364761}]",40.760179,-73.988029,Hell's Kitchen,NaN,NY,56d61e11498e4c3c305da704
9,Renaissance New York Times Square Hotel,Hotel,"Two Times Square, 714 Seventh Avenue At W. 48th Street",US,New York,United States,at W 48th St,192,"[Two Times Square, 714 Seventh Avenue At W. 48th Street (at W 48th St), New York, NY 10036, United States]","[{'label': 'display', 'lat': 40.75971, 'lng': -73.984643}]",40.759710,-73.984643,NaN,10036,NY,4a7d0598f964a5205aee1fe3
13,Night Hotel Times Square,Hotel,157 W 47th St,US,New York,United States,7th Avenue,256,"[157 W 47th St (7th Avenue), New York, NY 10036, United States]","[{'label': 'display', 'lat': 40.75908099999999, 'lng': -73.983701}]",40.759081,-73.983701,NaN,10036,NY,5145c277e4b0cec0809bad8c
8,Millennium Broadway Hotel,Hotel,145 W 44th St,US,New York,United States,at Broadway,272,"[145 W 44th St (at Broadway), New York, NY 10036, United States]","[{'label': 'display', 'lat': 40.757073, 'lng': -73.9848}]",40.757073,-73.984800,NaN,10036,NY,4a0215c6f964a5202a711fe3
17,Serafina Time Hotel,Italian Restaurant,224 W 49th St,US,New York,United States,8th Avenue,277,"[224 W 49th St (8th Avenue), New York, NY 10019, United States]","[{'label': 'display', 'lat': 40.76130213956465, 'lng': -73.98537892620858}]",40.761302,-73.985379,NaN,10019,NY,4b63389ef964a520986b2ae3
12,The Pearl Hotel,Hotel,233 W 49th St,US,New York,United States,btwn Broadway & 8th Ave.,284,"[233 W 49th St (btwn Broadway & 8th Ave.), New York, NY 10019, United States]","[{'label': 'display', 'lat': 40.76143050528299, 'lng': -73.98557585645281}]",40.761431,-73.985576,NaN,10019,NY,4cc5e3de06c254813e2e9f47
7,Kimpton Muse Hotel,Hotel,130 W 46th St,US,New York,United States,btwn 6th & 7th Ave,286,"[130 W 46th St (btwn 6th & 7th Ave), New York, NY 10036, United States]","[{'label': 'display', 'lat': 40.757808, 'lng': -73.983764}]",40.757808,-73.983764,NaN,10036,NY,4a9f2f6ff964a520d93c20e3
20,Night Hotel,Hotel,132 W 45th St,US,New York,United States,at Avenue of the Americas,293,"[132 W 45th St (at Avenue of the Americas), New York, NY 10036, United States]","[{'label': 'display', 'lat': 40.75729176565184, 'lng': -73.98413937207532}]",40.757292,-73.984139,NaN,10036,NY,44826b5af964a5201b341fe3


Let's save the query results to a local .CSV file.

Let's check the ratings of some the hotels in our dataset, starting with the closest one, the Paramount.

In [323]:
venue_id = '4adbaf34f964a520012a21e3' # Paramount Hotel ID
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION) 

In [324]:
result = requests.get(url).json()
print(result['response']['venue'].keys())
result['response']['venue']

dict_keys(['id', 'name', 'contact', 'location', 'canonicalUrl', 'categories', 'verified', 'stats', 'url', 'hasMenu', 'likes', 'dislike', 'ok', 'rating', 'ratingColor', 'ratingSignals', 'menu', 'allowMenuUrlEdit', 'beenHere', 'specials', 'photos', 'venuePage', 'reasons', 'description', 'storeId', 'page', 'hereNow', 'createdAt', 'tips', 'shortUrl', 'timeZone', 'listed', 'hours', 'pageUpdates', 'inbox', 'attributes', 'bestPhoto', 'colors'])


{'id': '4adbaf34f964a520012a21e3',
 'name': 'Paramount Hotel',
 'contact': {'phone': '2127645500',
  'formattedPhone': '(212) 764-5500',
  'twitter': 'nycparamount',
  'facebook': '101786016079',
  'facebookUsername': 'NYCParamount',
  'facebookName': 'Paramount Hotel Times Square New York'},
 'location': {'address': '235 W 46th St',
  'lat': 40.759568887065484,
  'lng': -73.98709530548786,
  'labeledLatLngs': [{'label': 'display',
    'lat': 40.759568887065484,
    'lng': -73.98709530548786}],
  'postalCode': '10036',
  'cc': 'US',
  'city': 'New York',
  'state': 'NY',
  'country': 'United States',
  'formattedAddress': ['235 W 46th St',
   'New York, NY 10036',
   'United States']},
 'canonicalUrl': 'https://foursquare.com/v/paramount-hotel/4adbaf34f964a520012a21e3',
 'categories': [{'id': '4bf58dd8d48988d1fa931735',
   'name': 'Hotel',
   'pluralName': 'Hotels',
   'shortName': 'Hotel',
   'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/travel/hotel_',
    'suffix': '.pn

In [325]:
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

6.1


That's not a very good rating, so let's try the citizenM Hotel which is only 332 meters away.

In [326]:
venue_id = '5093c236830214706abb75db' # citizenM Hotel ID
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION) 

In [327]:
result = requests.get(url).json()
print(result['response']['venue'].keys())
result['response']['venue']

dict_keys(['id', 'name', 'contact', 'location', 'canonicalUrl', 'categories', 'verified', 'stats', 'url', 'likes', 'dislike', 'ok', 'rating', 'ratingColor', 'ratingSignals', 'allowMenuUrlEdit', 'beenHere', 'specials', 'photos', 'reasons', 'storeId', 'page', 'hereNow', 'createdAt', 'tips', 'shortUrl', 'timeZone', 'listed', 'pageUpdates', 'inbox', 'attributes', 'bestPhoto', 'colors'])


{'id': '5093c236830214706abb75db',
 'name': 'citizenM Hotel New York Times Square',
 'contact': {'phone': '2124613638',
  'formattedPhone': '(212) 461-3638',
  'twitter': 'citizenm',
  'facebook': '173364980699',
  'facebookUsername': 'citizenMhotels',
  'facebookName': 'citizenM hotels'},
 'location': {'address': '218 W 50th St',
  'crossStreet': "50th & B'way",
  'lat': 40.761691026346256,
  'lng': -73.98495316332877,
  'labeledLatLngs': [{'label': 'display',
    'lat': 40.761691026346256,
    'lng': -73.98495316332877}],
  'postalCode': '10019',
  'cc': 'US',
  'city': 'New York',
  'state': 'NY',
  'country': 'United States',
  'formattedAddress': ["218 W 50th St (50th & B'way)",
   'New York, NY 10019',
   'United States']},
 'canonicalUrl': 'https://foursquare.com/v/citizenm-hotel-new-york-times-square/5093c236830214706abb75db',
 'categories': [{'id': '4bf58dd8d48988d1fa931735',
   'name': 'Hotel',
   'pluralName': 'Hotels',
   'shortName': 'Hotel',
   'icon': {'prefix': 'https:/

In [328]:
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

9.2


Wow, this looks like a great choice for our customers. Let's get the number of ratings for this hotel.

In [329]:
result['response']['venue']['ratingSignals']

412

Now let's get the number of tips.

That's a lot of ratings, so this seems like a good hotel for our customers. Let's get the hotel's tips from Foursquare.

In [330]:
result['response']['venue']['tips']['count']

83

In [331]:
tips = results['response']['tips']['items']

tip = results['response']['tips']['items'][0]
tip.keys()

KeyError: 'tips'

In [ ]:
pd.set_option('display.max_colwidth', -1)

tips_df = json_normalize(tips) # json normalize tips

# columns to keep
filtered_columns = ['text', 'agreeCount', 'disagreeCount', 'id', 'user.firstName', 'user.lastName', 'user.gender', 'user.id']
tips_filtered = tips_df.loc[:, filtered_columns]

# display tips
tips_filtered

Let's select the citizenM Hotel as our preferred hotel, and search for the best restaurants nearby.

## 5. Restaurants Near the Broadway Theatre District and citizenM Hotel <a name="restaurants-5"></a>

In [ ]:
# Latitude and Longitude of citizenM Hotel
latitude = 40.761691
longitude = -73.984953

In [ ]:
search_query = 'Restaurant'
radius = 500
print(search_query + ' .....OK!')

In [ ]:
# Set up the search url to query Foursquare
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)

In [ ]:
# Get the results as a JSON file
results = requests.get(url).json()
results

In [ ]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

In [ ]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
restaurants_filtered_df = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
restaurants_filtered_df['categories'] = restaurants_filtered_df.apply(get_category_type, axis=1)

# clean column names by keeping only last term
restaurants_filtered_df.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

restaurants_filtered_df

In [ ]:
restaurants_filtered_df.name

### Map of Restaurants Near the Broadway Theatre District <a name="rest-map"></a>

Let's map out the location of the 29 restaurants near the citizenM Hotel returned by our Foursquare query.

In [332]:
restaurants_map = folium.Map(location=[latitude, longitude], zoom_start=15) # generate map centred around the citizenM Hotel

# add a red circle marker to represent the citizenM Hotel
folium.features.CircleMarker(
    [latitude, longitude],
    radius=5,
    color='red',
    popup='citizenM Hotel',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(restaurants_map)

# add the restaurants as blue circle markers
for lat, lng, name in zip(restaurants_filtered_df['lat'],restaurants_filtered_df['lng'],restaurants_filtered_df['name']):
    label = '{}'.format(name)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
    popup=label,
    color='yellow',
    fill=True,
    fill_color='red',
    fill_opacity=0.7,
    parse_html=False).add_to(restaurants_map)

# display map
restaurants_map

**Restaurants within 500 meters of the citizenM Hotel, marked in red, returned by our Foursquare API query**

In [ ]:
Now that we have some nearby restaurants we can build out the details of our tour package.

## 6. Conclusions and Findings <a name="conclusions"></a>

We have researched the Broadway Theatre District in the Manhattan borough of New York City, and now have list of current shows and theatres where they are appearing.

We also have a list of hotels near the Richard Rodgers Theatre where Hamilton is continuing it's long rung. From this list, we have a recommended hotel, the citizenM with a 9.2 rating from over 400 users.

Finally, we have a list of restaurants close to our hotel we can build in our travel package.

Here is a report on the findings.

- [Current Broadway Productions and Associated Theatres](#current-productions)
- [Recommended Hotel](#recommend-hotel)
- [Nearby Restaurants](#nearby-rest)

### Current Broadway Productions and Associated Theatres <a name="current-productions"></a>

In [334]:
CurrentProductions_df = pd.read_csv('current_productions.csv')
CurrentProductions_df

,Theatre,Address,Capacity,CurrentProduction,Type
0,Al Hirschfeld Theatre,302 West 45th St.,1424,Moulin Rouge,Musical
1,Ambassador Theatre,219 West 49th St.,1125,Chicago,Musical
2,American Airlines Theatre,227 West 42nd St.,740,A Soldier's Play,Play
3,August Wilson Theatre,245 West 52nd St.,1228,Mean Girls,Musical
4,Belasco Theatre,111 West 44th St.,1018,Girl from the North Country,Musical
5,Bernard B. Jacobs Theatre,242 West 45th St.,1078,Company,Musical
6,Booth Theatre,222 West 45th St.,766,Who's Afraid of Virginia Woolf?,Play
7,Broadhurst Theatre,235 West 44th St.,1186,Jagged Little Pill,Musical
8,Broadway Theatre,1681 West 53rd St & Broadway,1761,West Side Story,Musical
9,Brooks Atkinson Theatre,256 West 47th St.,1094,Six,Musical


### Recommended Hotel <a name="recommend-hotel"></a>

The recommended hotel for our package is the 4-Star citizenM Hotel.

![citizenM](citizenM.jpg)

- **Address**: 218 W 50th St, New York, NY 10019
- **Reviews**: 4.6 stars, with 1,587 reviews on Google.
- **Room Rates**: as low as $169 a night
- **Features**: In walking distance of many of the Broadway Theatres, this hotel has the following amenities:

**From the citizenM website**:

Located just off the corner of Broadway and 50th street, our New York hotel is right in the heart of Midtown Manhattan, smack bang in the middle of the Theater District. A 5 minute walk from 5th Avenue, Madison Avenue, Central Park and Columbus Circle, 10-15 minutes away from the major museums, Rockefeller Center and the Empire State Building, and an easy subway or cab ride to anywhere else in the city, from Soho to the Meatpacking District – so if you’re looking for the coolest hotels New York has to offer, check out citizenM.

### Nearby Restaurants <a name="nearby-rest"></a>

The following 30 restaurants are within walking distance of the citizenM Hotel:

In [333]:
Restaurants_df4 = pd.read_csv('BroadwayRestClean.csv', index_col=False)
Restaurants_df4

,Restaurant,Category,Address,Distance
0,The View Restaurant & Lounge,Restaurant,1535 Broadway,54
1,Junior's Restaurant & Bakery,American Restaurant,1515 Broadway,78
2,Carmine's Italian Restaurant,Italian Restaurant,200 W 44th St,170
3,L'ybane Restaurant,Cocktail Bar,709 8th Ave,176
4,Smith's Bar & Restaurant,Bar,701 8th Ave,188
5,Ding BBQ and Hot Pot Restaurant,Hotpot Restaurant,234 W 48th St,194
6,Restaurant Row,Other Great Outdoors,W. 46th St.,254
7,O'Donoghues Pub & Restaurant,Bar,156 W 44th St,259
8,Junior's Restaurant,American Restaurant,1626 Broadway,279
9,Connolly's Pub & Restaurant,Irish Pub,121 W 45th St,324
